# Questions Per Language
This notebook intends to show how many stackoverflow questions exist for each programming language that has any considerable measure of use. 

# Step 0: Dependencies
This notebood expects you to already have the major python data science tools like numpy, pandas, and beautifulsoup. The author wrote it using anaconda, and it's recommneded that you use anaconda as well. Odds are if you have jupyter installed, you already have all the necessary tools.

# Step 1: Get a List of Programming Languages
We will use the [wikipedia page "list of programming languages"](https://en.wikipedia.org/wiki/List_of_programming_languages) as our major source. We will begin by downloading the HTML for that page and scraping it for names of programming languages.

In [1]:
WIKIPEDIA_LIST_OF_PROGRAMMING_LANGUAGES = 'https://en.wikipedia.org/wiki/List_of_programming_languages'

import requests
from bs4 import BeautifulSoup

with requests.get(WIKIPEDIA_LIST_OF_PROGRAMMING_LANGUAGES) as r:
    soup = BeautifulSoup(r.text, 'html.parser')

In [2]:
content = soup.find('div', {'id': 'mw-content-text'})
sections = content.find_all('div', {'class': 'columns'})

sections_combined = sections[0]
for i in range(1, len(sections)):
    sections_combined.append(sections[i])

# in this case, the following two lines are equivalent:

#lis = sections_combined.findChildren('li')
lis = sections_combined.find_all('li')

print(lis[0])

<li><a href="/wiki/A_Sharp_(.NET)" title="A Sharp (.NET)">A.NET</a></li>
